# Accessing Odds API to gather Player Props data

In [200]:
import requests
import json
import pandas as pd
import re
import numpy as np
from datetime import datetime, timedelta
import os

In [202]:
api_key_1 = os.getenv('Sport_Game_Odd_Key') # for the sportsGame API
api_key_2 = os.getenv('Odds_API_Key') # For the odds API

In [294]:
header = {'x-api-key': api_key_1}

In [296]:
today = datetime.today()
tomorrow = today + timedelta(days = 1)
yesterday = today - timedelta(days = 1)

In [297]:
response = requests.get('https://api.sportsgameodds.com/v1/events', params={
    'leagueID': 'NBA',
    'marketOddsAvailable': 'true',
    'limit': 30,
    'startsAfter': today,
    'startsBefore': tomorrow
}, headers = header)

data = response.json()

In [298]:
data

{'success': True,
 'data': [{'eventID': 'QCVm8kNUMMI5i7bYRk8G',
   'sportID': 'BASKETBALL',
   'leagueID': 'NBA',
   'type': 'match',
   'info': {'seasonWeek': 'NBA 24/25'},
   'teams': {'home': {'statEntityID': 'home',
     'names': {'nickname': 'Kings',
      'short': 'SAC',
      'location': 'Sacramento',
      'medium': 'Kings',
      'long': 'Sacramento Kings'},
     'teamID': 'SACRAMENTO_KINGS_NBA',
     'colors': {'secondary': '#8E9090',
      'primaryContrast': '#FFFFFF',
      'secondaryContrast': '#724C9F',
      'primary': '#724C9F'}},
    'away': {'statEntityID': 'away',
     'names': {'nickname': 'Lakers',
      'short': 'LAL',
      'location': 'Los Angeles',
      'medium': 'Lakers',
      'long': 'Los Angeles Lakers'},
     'teamID': 'LOS_ANGELES_LAKERS_NBA',
     'colors': {'secondary': '#FDB927',
      'primaryContrast': '#FFFFFF',
      'secondaryContrast': '#552582',
      'primary': '#552582'}}},
   'players': {'DEAARON_FOX_SACRAMENTO_KINGS_NBA': {'playerID': 'DEAA

In [300]:
length = len(data['data'])

In [301]:
length

12

In [302]:
data['data'][0]['teams']['away']

{'statEntityID': 'away',
 'names': {'nickname': 'Lakers',
  'short': 'LAL',
  'location': 'Los Angeles',
  'medium': 'Lakers',
  'long': 'Los Angeles Lakers'},
 'teamID': 'LOS_ANGELES_LAKERS_NBA',
 'colors': {'secondary': '#FDB927',
  'primaryContrast': '#FFFFFF',
  'secondaryContrast': '#552582',
  'primary': '#552582'}}

In [303]:
for value in data['data'][0]['players']:
    print(value)

DEAARON_FOX_SACRAMENTO_KINGS_NBA
AUSTIN_REAVES_LOS_ANGELES_LAKERS_NBA
RUI_HACHIMURA_LOS_ANGELES_LAKERS_NBA
KEEGAN_MURRAY_SACRAMENTO_KINGS_NBA
LEBRON_JAMES_LOS_ANGELES_LAKERS_NBA
MALIK_MONK_SACRAMENTO_KINGS_NBA
ANTHONY_DAVIS_LOS_ANGELES_LAKERS_NBA
MAX_CHRISTIE_LOS_ANGELES_LAKERS_NBA
DOMANTAS_SABONIS_SACRAMENTO_KINGS_NBA


In [305]:
data['data'][0]['players']

{'DEAARON_FOX_SACRAMENTO_KINGS_NBA': {'playerID': 'DEAARON_FOX_SACRAMENTO_KINGS_NBA',
  'firstName': "De'Aaron",
  'lastName': 'Fox',
  'name': "De'Aaron Fox",
  'teamID': 'SACRAMENTO_KINGS_NBA'},
 'AUSTIN_REAVES_LOS_ANGELES_LAKERS_NBA': {'playerID': 'AUSTIN_REAVES_LOS_ANGELES_LAKERS_NBA',
  'firstName': 'Austin',
  'lastName': 'Reaves',
  'name': 'Austin Reaves',
  'teamID': 'LOS_ANGELES_LAKERS_NBA'},
 'RUI_HACHIMURA_LOS_ANGELES_LAKERS_NBA': {'playerID': 'RUI_HACHIMURA_LOS_ANGELES_LAKERS_NBA',
  'firstName': 'Rui',
  'lastName': 'Hachimura',
  'name': 'Rui Hachimura',
  'teamID': 'LOS_ANGELES_LAKERS_NBA'},
 'KEEGAN_MURRAY_SACRAMENTO_KINGS_NBA': {'playerID': 'KEEGAN_MURRAY_SACRAMENTO_KINGS_NBA',
  'firstName': 'Keegan',
  'lastName': 'Murray',
  'name': 'Keegan Murray',
  'teamID': 'SACRAMENTO_KINGS_NBA'},
 'LEBRON_JAMES_LOS_ANGELES_LAKERS_NBA': {'playerID': 'LEBRON_JAMES_LOS_ANGELES_LAKERS_NBA',
  'firstName': 'LeBron',
  'lastName': 'James',
  'name': 'LeBron James',
  'teamID': 'LOS

In [328]:
def extract_player_point_props(json_data, i):
    player_props = {}
    pattern = r"points-([A-Z_]+)_NBA-game-ou-(over|under)"
    three_word_teams = ["new", "golden", "los", "oklahoma", "san", "portland"]
        
    for key, value in json_data['odds'].items():
        match = re.match(pattern, key)
        if match:
            full_name = match.group(1).replace('_', ' ').lower()
            over_under = match.group(2).lower()
            
            # Split the full name into parts
            name_parts = full_name.split()
            # Check if the third-to-last word is in the list of three-word team prefixes
            if len(name_parts) > 3 and name_parts[-3] in three_word_teams:
                player_name = ' '.join(name_parts[:-3])  # Everything except the last three words
                team_name = ' '.join(name_parts[-3:])    # Last three words
            else:
                player_name = ' '.join(name_parts[:-2])  # Everything except the last two words
                team_name = ' '.join(name_parts[-2:])    # Last two words
            
            if player_name not in player_props:
                player_props[player_name] = {
                    'team': team_name,
                    'line': value['overUnder'],
                    'over': {'odds': None, 'book_line': None},
                    'under': {'odds': None, 'book_line': None}
                }
            try:
                if value['sideID'].lower() == 'over':
                    player_props[player_name]['over'] = {
                        'book_line': value['bookOdds']
                    }
                elif value['sideID'].lower() == 'under':
                    player_props[player_name]['under'] = {
                        'book_line': value['bookOdds']
                    }
            except:
                print('player not playing')
            print(player_name)
    for value in json_data['players']:
        
        name = json_data['players'][value]['name'].lower().replace('-', '').replace('.', '').replace(' ii', '').replace("'", "").replace('ć', 'c')\
        .replace('č', 'c')
        
        if name == "josh giddey":
            name = "joshua giddey"
        if name == 'scotty pippen jr':
            name = "scotty pippen"
        if name == 'andre jackson jr':
            name = "andre jackson"
        if name == 'jesse edwards':
            name = 'anthony edwards'
        if name == 'trey murphyi':
            name = 'trey murphy'
        if name == 'lindy watersi':
            name = 'lindy waters'
        if name == 'pj washington jr':
            name = 'pj washington'
        if name == "ethan thompson":
            name = "klay thompson"
        if name == "derrick jones jr":
            name = "derrick jones"
        
        if player_name == 'jeenathan williams':
            continue
        if name == 'mason jones':
            name = 'colby jones'
        
        if name not in player_props:
            player_props[name] = {
                'team': team_name,
                'line': None,
                'over': {'odds': None, 'book_line': None},
                'under': {'odds': None, 'book_line': None}
            }
        print(json_data['players'][value]['teamID'], name)

        if json_data['players'][value]['teamID'] == data['data'][i]['teams']['home']['teamID']:
            player_props[name]['opp'] = data['data'][i]['teams']['away']['names']['short']
        else:
            player_props[name]['opp'] = data['data'][i]['teams']['home']['names']['short']


    return player_props

In [330]:
game_data = []
for i in range(length):
    d = extract_player_point_props(data['data'][i], i)
    game_data.append(d)

anthony davis
anthony davis
max christie
lebron james
lebron james
max christie
deaaron fox
rui hachimura
rui hachimura
austin reaves
deaaron fox
keegan murray
malik monk
austin reaves
malik monk
keegan murray
domantas sabonis
domantas sabonis
SACRAMENTO_KINGS_NBA deaaron fox
LOS_ANGELES_LAKERS_NBA austin reaves
LOS_ANGELES_LAKERS_NBA rui hachimura
SACRAMENTO_KINGS_NBA keegan murray
LOS_ANGELES_LAKERS_NBA lebron james
SACRAMENTO_KINGS_NBA malik monk
LOS_ANGELES_LAKERS_NBA anthony davis
LOS_ANGELES_LAKERS_NBA max christie
SACRAMENTO_KINGS_NBA domantas sabonis
duncan robinson
tyler herro
tyler herro
jalen suggs
tristan da silva
jimmy butler
jimmy butler
goga bitadze
tristan da silva
haywood highsmith
haywood highsmith
wendell carter jr
jalen suggs
duncan robinson
wendell carter jr
bam adebayo
bam adebayo
goga bitadze
MIAMI_HEAT_NBA bam adebayo
MIAMI_HEAT_NBA tyler herro
ORLANDO_MAGIC_NBA jalen suggs
MIAMI_HEAT_NBA duncan robinson
ORLANDO_MAGIC_NBA tristan da silva
ORLANDO_MAGIC_NBA goga 

### Below is commented out since only needed once

In [333]:
game_data

[{'anthony davis': {'team': 'los angeles lakers',
   'line': '27.5',
   'over': {'book_line': '-115'},
   'under': {'book_line': '-115'},
   'opp': 'SAC'},
  'max christie': {'team': 'los angeles lakers',
   'line': '7.5',
   'over': {'book_line': '+100'},
   'under': {'book_line': '-136'},
   'opp': 'SAC'},
  'lebron james': {'team': 'los angeles lakers',
   'line': '24.5',
   'over': {'book_line': '-118'},
   'under': {'book_line': '-113'},
   'opp': 'SAC'},
  'deaaron fox': {'team': 'sacramento kings',
   'line': '25.5',
   'over': {'book_line': '-120'},
   'under': {'book_line': '-110'},
   'opp': 'LAL'},
  'rui hachimura': {'team': 'los angeles lakers',
   'line': '12.5',
   'over': {'book_line': '-107'},
   'under': {'book_line': '-124'},
   'opp': 'SAC'},
  'austin reaves': {'team': 'los angeles lakers',
   'line': '17.5',
   'over': {'book_line': '-127'},
   'under': {'book_line': '-105'},
   'opp': 'SAC'},
  'keegan murray': {'team': 'sacramento kings',
   'line': '12.5',
   '

df = pd.DataFrame()
df = df.reindex(range(1000))
num_rows = len(df)

##### Calculate the start date 
start_date = datetime.now().date() - timedelta(days = 400)

##### Create a date range
date_range = pd.date_range(start=start_date, periods=num_rows)

##### Add the 'Date' column to your DataFrame
df['Date'] = date_range

##### Format the date as MM/DD if you prefer
df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')

In [336]:
df = pd.read_csv('prop_table.csv', index_col=0)

C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\3114025419.py:1: DtypeWarning: Columns (502,504,506,508,511,514,516,519,521,524,526,529,532,535,538,540,543,546,548,551,553,556,559,562,565,567,570,572,574,576,578,581,583,585,587,590,593,595,598,601,603,605,608,610,613,615,618,620,622,624,627,629,632,635,638,641,644,646,648,651,653,655,658,660,663,665,667,669,671,674,677,679,682,685,688,690,692,695,698,700,702,705,707,710,712,714,717,720,723,725,728,730,732,735,737,739,742,744,746,748,751,753,756,758,761,763,765,767,770,773,776,779,782,784,786,789,792,794,796,798,801,804,807,809,812,814,817,819,822,824,827,830,832,835,837,839,841,844,847,849,851,853,856,859,862,864,867,870,872,874,877,880,882,885,888,890,893,895,898,900,903,906,908,911,913,915,917,920,923,926,928,931,933,936,938,940,942,945,948,951,954,957,959,961,963,965,968,970,973,975,978,981,983,986,988,990,993,995,997,1000,1002,1005,1007,1010,1012,1014,1016,1018,1020,1022,1025,1028,1031,1033,1035,1049,1050,1051,1052,1053,1054,1055,

In [337]:
game_data

[{'anthony davis': {'team': 'los angeles lakers',
   'line': '27.5',
   'over': {'book_line': '-115'},
   'under': {'book_line': '-115'},
   'opp': 'SAC'},
  'max christie': {'team': 'los angeles lakers',
   'line': '7.5',
   'over': {'book_line': '+100'},
   'under': {'book_line': '-136'},
   'opp': 'SAC'},
  'lebron james': {'team': 'los angeles lakers',
   'line': '24.5',
   'over': {'book_line': '-118'},
   'under': {'book_line': '-113'},
   'opp': 'SAC'},
  'deaaron fox': {'team': 'sacramento kings',
   'line': '25.5',
   'over': {'book_line': '-120'},
   'under': {'book_line': '-110'},
   'opp': 'LAL'},
  'rui hachimura': {'team': 'los angeles lakers',
   'line': '12.5',
   'over': {'book_line': '-107'},
   'under': {'book_line': '-124'},
   'opp': 'SAC'},
  'austin reaves': {'team': 'los angeles lakers',
   'line': '17.5',
   'over': {'book_line': '-127'},
   'under': {'book_line': '-105'},
   'opp': 'SAC'},
  'keegan murray': {'team': 'sacramento kings',
   'line': '12.5',
   '

In [340]:
for d in game_data:
    print(d)

{'anthony davis': {'team': 'los angeles lakers', 'line': '27.5', 'over': {'book_line': '-115'}, 'under': {'book_line': '-115'}, 'opp': 'SAC'}, 'max christie': {'team': 'los angeles lakers', 'line': '7.5', 'over': {'book_line': '+100'}, 'under': {'book_line': '-136'}, 'opp': 'SAC'}, 'lebron james': {'team': 'los angeles lakers', 'line': '24.5', 'over': {'book_line': '-118'}, 'under': {'book_line': '-113'}, 'opp': 'SAC'}, 'deaaron fox': {'team': 'sacramento kings', 'line': '25.5', 'over': {'book_line': '-120'}, 'under': {'book_line': '-110'}, 'opp': 'LAL'}, 'rui hachimura': {'team': 'los angeles lakers', 'line': '12.5', 'over': {'book_line': '-107'}, 'under': {'book_line': '-124'}, 'opp': 'SAC'}, 'austin reaves': {'team': 'los angeles lakers', 'line': '17.5', 'over': {'book_line': '-127'}, 'under': {'book_line': '-105'}, 'opp': 'SAC'}, 'keegan murray': {'team': 'sacramento kings', 'line': '12.5', 'over': {'book_line': '-119'}, 'under': {'book_line': '-109'}, 'opp': 'LAL'}, 'malik monk': 

In [342]:
date = datetime.now().date().strftime('%Y/%m/%d') 
for d in game_data:
    for player in d:
        print(player)
        
        # Create a column for the player if it doesn't exist
        if f'{player}_line' not in df.columns:
            df[f'{player}_line'] = np.nan
        if f'{player}_under' not in df.columns:
            df[f'{player}_under'] = np.nan
        if f'{player}_over' not in df.columns:
            df[f'{player}_over'] = np.nan
        
        
        # Find the first NaN value in the player's column
    
        print(date)
        # Assign the new value to the next empty row
        df.loc[df['date'] == date, f'{player}_line'] = d[player]['line']
        df.loc[df['date'] == date, f'{player}_over'] = d[player]['over']['book_line']
        df.loc[df['date'] == date, f'{player}_under'] = d[player]['under']['book_line']
        print(d[player])
        if player == "jeenathan williams":
            continue
        df.loc[df['date'] == date, f'{player}_todays_game'] = d[player]['opp']

anthony davis
2024/12/21
{'team': 'los angeles lakers', 'line': '27.5', 'over': {'book_line': '-115'}, 'under': {'book_line': '-115'}, 'opp': 'SAC'}
max christie
2024/12/21
{'team': 'los angeles lakers', 'line': '7.5', 'over': {'book_line': '+100'}, 'under': {'book_line': '-136'}, 'opp': 'SAC'}
lebron james
2024/12/21
{'team': 'los angeles lakers', 'line': '24.5', 'over': {'book_line': '-118'}, 'under': {'book_line': '-113'}, 'opp': 'SAC'}
deaaron fox
2024/12/21
{'team': 'sacramento kings', 'line': '25.5', 'over': {'book_line': '-120'}, 'under': {'book_line': '-110'}, 'opp': 'LAL'}
rui hachimura
2024/12/21
{'team': 'los angeles lakers', 'line': '12.5', 'over': {'book_line': '-107'}, 'under': {'book_line': '-124'}, 'opp': 'SAC'}
austin reaves
2024/12/21
{'team': 'los angeles lakers', 'line': '17.5', 'over': {'book_line': '-127'}, 'under': {'book_line': '-105'}, 'opp': 'SAC'}
keegan murray
2024/12/21
{'team': 'sacramento kings', 'line': '12.5', 'over': {'book_line': '-119'}, 'under': {'b

C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27.5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_line'] = d[player]['line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-115' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_over'] = d[player]['over']['book_line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-115' has dtype incompatible with float64, please explicitly cast to a comp

{'team': 'cleveland cavaliers', 'line': '23.5', 'over': {'book_line': '-112'}, 'under': {'book_line': '-118'}, 'opp': 'PHI'}
tyrese maxey
2024/12/21
{'team': 'cleveland cavaliers', 'line': None, 'over': {'odds': None, 'book_line': None}, 'under': {'odds': None, 'book_line': None}, 'opp': 'CLE'}
paul george
2024/12/21
{'team': 'cleveland cavaliers', 'line': None, 'over': {'odds': None, 'book_line': None}, 'under': {'odds': None, 'book_line': None}, 'opp': 'CLE'}
kelly oubre jr
2024/12/21
{'team': 'cleveland cavaliers', 'line': None, 'over': {'odds': None, 'book_line': None}, 'under': {'odds': None, 'book_line': None}, 'opp': 'CLE'}
caleb martin
2024/12/21
{'team': 'cleveland cavaliers', 'line': None, 'over': {'odds': None, 'book_line': None}, 'under': {'odds': None, 'book_line': None}, 'opp': 'CLE'}
derrick white
2024/12/21
{'team': 'boston celtics', 'line': '16.5', 'over': {'book_line': '-110'}, 'under': {'book_line': '-120'}, 'opp': 'CHI'}
coby white
2024/12/21
{'team': 'chicago bulls

C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-120' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_under'] = d[player]['under']['book_line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '33.5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_line'] = d[player]['line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-116' has dtype incompatible with float64, please explicitly cast to a co

{'team': 'milwaukee bucks', 'line': '12.5', 'over': {'book_line': '-110'}, 'under': {'book_line': '-120'}, 'opp': 'WAS'}
giannis antetokounmpo
2024/12/21
{'team': 'milwaukee bucks', 'line': '33.5', 'over': {'book_line': '-116'}, 'under': {'book_line': '-114'}, 'opp': 'WAS'}
alexandre sarr
2024/12/21
{'team': 'washington wizards', 'line': '11.5', 'over': {'book_line': '-116'}, 'under': {'book_line': '-114'}, 'opp': 'MIL'}
bilal coulibaly
2024/12/21
{'team': 'washington wizards', 'line': '13.5', 'over': {'book_line': '-118'}, 'under': {'book_line': '-110'}, 'opp': 'MIL'}
aj green
2024/12/21
{'team': 'washington wizards', 'line': None, 'over': {'odds': None, 'book_line': None}, 'under': {'odds': None, 'book_line': None}, 'opp': 'WAS'}
andre jackson
2024/12/21
{'team': 'washington wizards', 'line': None, 'over': {'odds': None, 'book_line': None}, 'under': {'odds': None, 'book_line': None}, 'opp': 'WAS'}
damian lillard
2024/12/21
{'team': 'washington wizards', 'line': None, 'over': {'odds':

C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-111' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_under'] = d[player]['under']['book_line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_line'] = d[player]['line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-105' has dtype incompatible with float64, please explicitly cast to a co

In [344]:
df = df.loc[:, ~df.columns.duplicated()]
df.shape

(1000, 1626)

In [346]:
df.columns = df.columns.str.lower()

In [348]:
# Define the patterns to match
patterns = ["new", "golden", "los", "oklahoma", "san", "portland"]
suffixes = ["over", "under", "line"]

# Create a regex pattern
pattern = '|'.join([f"{p}.*({s})" for p in patterns for s in suffixes])

# Drop columns that match the pattern
df_filtered = df.drop(columns=df.filter(regex=pattern).columns)

In [350]:
df_filtered.shape

(1000, 1626)

In [352]:
df_filtered.to_csv('prop_table.csv')

In [353]:
url = "https://api.sportsgameodds.com/v1/account/usage"

headers = header

response = requests.get(url, headers=headers)

print(response.json())

{'success': True, 'data': {'keyID': 'b8d294d968b4d0f753d53f4a9e8173952107b1a17338a758b68f98f2ab887092', 'customerID': 'cus_RAwtHswGZQdPPj', 'isActive': True, 'rateLimits': {'per-second': {'maxRequestsPerInterval': 'unlimited', 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 'n/a', 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': 'n/a'}, 'per-minute': {'maxRequestsPerInterval': 10, 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 1, 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': '2024-12-21T13:49:24.455Z'}, 'per-hour': {'maxRequestsPerInterval': 'unlimited', 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 'n/a', 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': 'n/a'}, 'per-day': {'maxRequestsPerInterval': 'unlimited', 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 'n/a', 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': 'n/a'}, 'per-month': {'maxRequestsPerInterval': 'unl